<a href="https://colab.research.google.com/github/Anshuman22coder/ML-projects/blob/main/PROJECT_STOCK_PRICE_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install yfinance

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn


In [ ]:
import tensorflow as tf
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
ticker_symbol = "AAPL"     # stock name...
start_date = "2010-01-01"
end_date = "2023-01-01"

ticker=yf.Ticker(ticker_symbol)
historical_data=ticker.history(start=start_date,end=end_date)  #historical data is a data frame...
historical_data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-12-23 00:00:00-05:00,129.700134,131.186158,128.432062,130.631378,63814900,0.0,0.0
2022-12-27 00:00:00-05:00,130.155832,130.185551,127.520614,128.818405,69007800,0.0,0.0
2022-12-28 00:00:00-05:00,128.461759,129.809087,124.697171,124.865585,85438400,0.0,0.0
2022-12-29 00:00:00-05:00,126.797421,129.264217,126.539849,128.402328,75703700,0.0,0.0
2022-12-30 00:00:00-05:00,127.213519,128.739163,126.242647,128.719345,77034200,0.0,0.0


In [ ]:
#extracting relevant features
# create lag features  ___feature engineering>>>
historical_data['Prev_Close']=historical_data['Close'].shift(1)
historical_data['Prev_Volume']=historical_data['Volume'].shift(1)
historical_data['Prev_Open']=historical_data['Open'].shift(1)
historical_data['Prev_High']=historical_data['High'].shift(1)
historical_data['Prev_Low']=historical_data['Low'].shift(1)

# Drop rows with NaN values
historical_data.dropna(inplace=True)
features=historical_data[['Open','High','Low','Volume','Prev_Close','Prev_Volume','Prev_Open','Prev_High','Prev_Low']]
#features.head()  # features is also a dataframe...
#type(features)

In [ ]:
scaler=MinMaxScaler()
scaled_features=scaler.fit_transform(features)
#print(scaled_features)
#convert scaled data to DataFrame
scaled_df=pd.DataFrame(scaled_features,columns=features.columns)
#compare previous vs after scaling ::
#print(features.head())
print(scaled_df.head())

       Open      High       Low    Volume  Prev_Close  Prev_Volume  Prev_Open  \
0  0.003851  0.003390  0.004064  0.307026    0.003818     0.248420   0.003649   
1  0.003813  0.003328  0.003622  0.280076    0.003882     0.307026   0.003851   
2  0.003358  0.002769  0.003322  0.239427    0.003289     0.280076   0.003813   
3  0.003106  0.002769  0.003324  0.223434    0.003221     0.239427   0.003358   
4  0.003540  0.002942  0.003216  0.231354    0.003465     0.223434   0.003106   

   Prev_High  Prev_Low  
0   0.003202  0.003910  
1   0.003390  0.004064  
2   0.003328  0.003622  
3   0.002769  0.003322  
4   0.002769  0.003324  


In [ ]:
#split the data into training and testing sets::
target=historical_data['Close']
#print(target.head())
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [ ]:
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training features shape: (2289, 9)
Testing features shape: (982, 9)
Training target shape: (2289,)
Testing target shape: (982,)


In [ ]:
# Define the MLP model
model = tf.keras.Sequential([    #input_size is signifying the no of features to be provided..
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

##optimizer="adam" is used to provide the wwights to the model,, ans loss is  signified as mean_squared_error

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 236654895824896.0000 - val_loss: 6178290532352.0000
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3052292800512.0000 - val_loss: 401172856832.0000
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 551900807168.0000 - val_loss: 325170954240.0000
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 322928312320.0000 - val_loss: 66407325696.0000
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 95608758272.0000 - val_loss: 134725795840.0000
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 221368303616.0000 - val_loss: 81944092672.0000
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 236720078848.0000 - val_loss: 80677380096.0000
Epoch 8/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 106042056704.0000 - val_loss: 61741342720.0000
Epoch 9/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 166076825600.0000 - val_loss: 178409652224.0000
Epoch 10/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [ ]:
#evaluate the model based on test data set
loss=model.evaluate(X_test,y_test,verbose=1)
print("Test loss: ,",loss)
predictions = model.predict(X_test)
predictions_df = pd.DataFrame(predictions, columns=['Predicted_Close'])
print(predictions_df.head())

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 17087396864.0000 
Test loss: , 15259367424.0
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
   Predicted_Close
0    -17269.996094
1    -82688.000000
2    107814.000000
3    -40345.996094
4     -8393.996094


In [ ]:
from sklearn.metrics import mean_squared_error
# Calculate the Mean Squared Error
mse=mean_squared_error(y_test,predictions)
# Display the Mean Squared Error
print("Mean Squared Error:",mse)


#This number represents the MSE,
#where a lower value indicates that the predicted stock prices \
#are close to the actual prices.








Mean Squared Error: 15259368365.576437


In [ ]:
# for predicting the closing price for a set of new data ,,::



#new data as a dictionary
new_data_dict = {
    'Open': [120.5, 121.8, 123.0],
    'Prev_Open': [130.5, 121.8, 133.0],
    'High': [122.0, 122.5, 124.0],
    'Prev_High': [122.0, 122.5, 124.0],
    'Low': [119.8, 120.2, 122.0],
    'Prev_Low': [199.8, 120.2, 222.0],
    'Prev_Volume': [50000, 52000, 54000],
    'Volume': [50000, 52000, 54000],
     'Prev_Close': [120.0, 121.5, 122.0]
}

# Convert the new data dictionary to a DataFrame
newTest_dict={"Close" :[121.5, 122.0, 123.5]}
new_data_df2=pd.DataFrame(newTest_dict)
scaler1=MinMaxScaler()
scaled_features2=scaler1.fit_transform(new_data_df2)#test data or the correct result of the next day ...
scaled_df2=pd.DataFrame(scaled_features2,columns=new_data_df2.columns)




#convert scaled data to DataFrame   #  these are of the new test data ...

new_data_df = pd.DataFrame(new_data_dict)
#scaling of the new dataset
scaler=MinMaxScaler()
scaled_features=scaler.fit_transform(new_data_df)
#convert scaled data to DataFrame
scaled_df=pd.DataFrame(scaled_features,columns=new_data_df.columns)
#print(scaled_features
#print(new_data_df.head())

#new_data_scaled = scaler.transform(new_data_df)
#scaled_df=pd.DataFrame(new_data_scaled,columns=features.columns)

# Make predictions on the new data
new_predictions = model.predict(scaled_df)
#predictions_df = pd.DataFrame(new_predictions, columns=['Predicted_Close'])
#print(predictions_df.head())

#unscaling the scaled prediction output/result;;;;
unscaled_predictions = scaler1.inverse_transform(new_predictions)
unscaled_df = pd.DataFrame(unscaled_predictions, columns=['Predicted_Close'])
print(unscaled_df.head())

from sklearn.metrics import mean_squared_error
# Calculate the Mean Squared Error
mse=mean_squared_error(new_predictions,scaled_df2)
# Display the Mean Squared Error
print("Mean Squared Error:",mse)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
   Predicted_Close
0       121.362541
1       121.179985
2       121.518135
Mean Squared Error: 0.3849266396380202
